# Project Part 3

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://www.kaggle.com/code/dereckhelms/cs39aa-project-part-2)

Colab link is a work in progress

## 1. Creation of Feed-Forward Model
Although I do not believe a LTSM model is the most appropriate for this set of structured data, I want to construct one to see how it behaves in relation to another deep learning model. 

In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load the dataset
df = pd.read_csv('insurance.csv')

# Extract features and target variable
features = df[['age', 'bmi', 'children', 'smoker', 'region']]
target = df['charges']

# Explicitly create a copy of the DataFrame to avoid SettingWithCopyWarning
features = features.copy()

# Convert categorical features to one-hot encoding
label_encoder = LabelEncoder()
features['smoker'] = label_encoder.fit_transform(features['smoker'])
features = pd.get_dummies(features, columns=['region'])

# Standardize numerical features
scaler = StandardScaler()
features[['age', 'bmi', 'children']] = scaler.fit_transform(features[['age', 'bmi', 'children']])

# Ensure all data is of numeric type
features = features.astype('float32')
target = target.astype('float32')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Define the model architecture
class InsuranceModel(nn.Module):
    def __init__(self, input_size):
        super(InsuranceModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # Adjust the number of hidden units as needed
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)  # Output is a single number for insurance charge prediction

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Create a model instance
input_size = X_train_tensor.shape[1]
model = InsuranceModel(input_size)

# Set up the loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust the learning rate as needed

# Training loop
num_epochs = 1000  # Adjust as needed
for epoch in range(num_epochs):
    # Forward pass
    predictions = model(X_train_tensor)
    
    # Compute the loss
    loss = criterion(predictions, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print training information
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the test set
with torch.no_grad():
    model.eval()
    test_predictions = model(X_test_tensor)
    test_loss = criterion(test_predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

# Print some predicted versus actual values
with torch.no_grad():
    model.eval()
    sample_indices = np.random.choice(len(X_test_tensor), 5, replace=False)
    for i in sample_indices:
        prediction = model(X_test_tensor[i])
        print(f'Sample {i + 1}: Predicted={prediction.item():.4f}, Actual={y_test_tensor[i].item():.4f}')



ModuleNotFoundError: No module named 'torch'